<a href="https://colab.research.google.com/github/ravi-gopalan/ipl_analysis/blob/master/ipl_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!ls -la

total 16
drwxr-xr-x 1 root root 4096 May 31 16:17 .
drwxr-xr-x 1 root root 4096 Jun  4 04:28 ..
drwxr-xr-x 1 root root 4096 May 31 16:17 .config
drwxr-xr-x 1 root root 4096 May 31 16:17 sample_data


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
cd /root

/root


In [0]:
!mkdir .kaggle

In [0]:
!cp '/content/gdrive/My Drive/Colab_stuff/kaggle.json' /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

In [6]:
cd /content

/content


In [0]:
!mkdir ipl

In [8]:
cd /content/ipl

/content/ipl


In [9]:
!kaggle datasets download -d divjain/ipl_data_till_2019

  0% 0.00/1.32M [00:00<?, ?B/s]
100% 1.32M/1.32M [00:00<00:00, 87.5MB/s]


In [10]:
!ls -la

total 1364
drwxr-xr-x 2 root root    4096 Jun  4 04:45 .
drwxr-xr-x 1 root root    4096 Jun  4 04:44 ..
-rw-r--r-- 1 root root 1385168 Jun  4 04:45 ipl_data_till_2019.zip


In [11]:
!unzip ipl_data_till_2019.zip

Archive:  ipl_data_till_2019.zip
  inflating: innings_data.csv        
  inflating: match_data.csv          


In [12]:
!ls -la

total 18080
drwxr-xr-x 2 root root     4096 Jun  4 04:45 .
drwxr-xr-x 1 root root     4096 Jun  4 04:44 ..
---------- 1 root root 16937599 May  5 08:51 innings_data.csv
-rw-r--r-- 1 root root  1385168 Jun  4 04:45 ipl_data_till_2019.zip
---------- 1 root root   173805 May  5 08:51 match_data.csv


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
df_innings = pd.read_csv('innings_data.csv',sep=',')

In [20]:
df_innings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176573 entries, 0 to 176572
Data columns (total 20 columns):
id                    176573 non-null int64
season                176573 non-null int64
batsman               176573 non-null object
bowler                176573 non-null object
innings               176573 non-null object
non_striker           176573 non-null object
replacements          30 non-null object
bowled_over           176573 non-null float64
batsman_team          176573 non-null object
player_out            176573 non-null object
fielder_caught_out    176573 non-null object
type_out              176573 non-null object
extras_wides          176573 non-null int64
extras_legbyes        176573 non-null int64
extras_noballs        176573 non-null int64
extras_byes           176573 non-null int64
extras_penalty        176573 non-null int64
total_extras_runs     176573 non-null int64
batsman_runs          176573 non-null int64
total_runs            176573 non-null int64
dt

In [25]:
df_innings.groupby(['batsman_team','batsman'],axis=1)['batsman_runs']